# Data Mining Project II

## Imports

In [341]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from collections import Counter
from enum import IntEnum
from sklearn.ensemble import HistGradientBoostingRegressor, HistGradientBoostingClassifier
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.multioutput import MultiOutputClassifier
from sklearn.metrics import r2_score, root_mean_squared_error, mean_absolute_error, mean_squared_error, accuracy_score, classification_report
from xgboost import XGBClassifier, XGBRFClassifier
from sklearn.utils.class_weight import compute_class_weight
from sklearn.neighbors import KNeighborsClassifier
import seaborn as sns
from sklearn.cluster import KMeans
from kneed import KneeLocator
from sklearn.feature_selection import chi2
from sklearn.feature_selection import SelectKBest
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.svm import SVC
from imblearn.over_sampling import SMOTE

## Data preparation

In [342]:
attributes = {
    "WEIGHTLBTC_A": "weight", #pounds
    "HEIGHTTC_A": "height", #inches
    "PREGNOW_A": "pregnant",
    "AGEP_A": "age",
    "SEX_A": "gender",
    "EDUCP_A": "education",
    "MARITAL_A": "single",
    "HOUTENURE_A": "owns_home",
    "PARSTAT_A": "parent",
    "DEPFREQ_A": "depression",
    "ANXFREQ_A": "anxiety",
    "POVRATTC_A": "poverty",
    "PHSTAT_A": "health_status",
    "LSATIS4_A": "life_sat",
    #"DRK12MN_A": "alcohol",
    #"SLPHOURS_A": "sleep",
    "NOTCOV_A": "insurance",
    "URBRRL": "living_area",
    "REGION": "region",
    "CANEV_A": "cancer",
    "HYPEV_A": "hypertension",
    "CHLEV_A": "cholesterol",
    "ASEV_A": "asthma",
    "RACEALLP_A": "race",
    "PAYWORRY_A": "medical_bill_worry"
}

dataframes = []
for i in [2, 3]:
    df = pd.read_csv(f"../data/adult2{i}.csv", sep=",")
    dataframes.append(df)
    print(f"YEAR 202{i}\nfeatures: {len(df.columns)}\nentries: {len(df)}")
    for attr in attributes.keys():
        try:
            (f"{attr}: {df[attr].unique()[:(min(10, len(df[attr].unique())))]}")
        except:
            print(f"missing {attr} for 202{i}")
df = pd.concat(dataframes)
df = df.loc[:, list(attributes.keys())]
df.rename(mapper=attributes, inplace=True, axis=1)
list(df.columns)

YEAR 2022
features: 637
entries: 27651
YEAR 2023
features: 647
entries: 29522


['weight',
 'height',
 'pregnant',
 'age',
 'gender',
 'education',
 'single',
 'owns_home',
 'parent',
 'depression',
 'anxiety',
 'poverty',
 'health_status',
 'life_sat',
 'insurance',
 'living_area',
 'region',
 'cancer',
 'hypertension',
 'cholesterol',
 'asthma',
 'race',
 'medical_bill_worry']

In [343]:
df = df[df["weight"] <= 299]
df = df[df["height"] <= 76]
df = df[((df["pregnant"] == 2) | (df["gender"] == 1))]
df = df[df["age"] <= 84]
df = df[df["gender"] <= 2]
df = df[df["education"] <= 10]
df = df[df["single"] <= 3]
df = df[df["owns_home"] <= 3]
df = df[df["parent"] <= 3]
df = df[df["depression"] <= 5]
df = df[df["anxiety"] <= 5]
df = df[df["poverty"] < 11]
df = df[df["health_status"] < 6]
df = df[df["life_sat"] < 5]
#df = df[df["alcohol"] < 366]
#df = df[df["sleep"] < 25]
df = df[df["insurance"] <= 2]
# living area is good as it is
# region too
df = df[df["cancer"] <= 2]
df = df[df["hypertension"] <= 2]
df = df[df["cholesterol"] <= 2]
df = df[df["asthma"] <= 2]
df = df[df["race"] <= 6]
df = df[df["medical_bill_worry"] <= 3]

In [344]:
# BMI calculation
POUND_TO_KG = 0.453592
INCH_TO_M = 0.0254
df["weight"] = df["weight"] * POUND_TO_KG
df["height"] = df["height"] * INCH_TO_M
df["bmi"] = df["weight"] / df["height"] ** 2

del df["weight"]
del df["height"]
del df["pregnant"]

In [345]:
def get_optimal_cluster_number(df: pd.DataFrame, feature: str, k_number: int) -> int:
    inertia = []
    k_values = np.arange(1, k_number+1)

    for k in k_values:
        kmeans = KMeans(n_clusters=k, random_state=42)
        kmeans.fit(df[[feature]])
        inertia.append(kmeans.inertia_)

    knee_locator = KneeLocator(k_values, inertia, curve='convex', direction='decreasing')
    k = knee_locator.knee


    """fig, ax = plt.subplots(1, figsize=(8, 6))
    ax.plot(k_values, inertia, marker='o')
    ax.set(xlabel='Number of clusters (k)', ylabel='Inertia', title=f'Elbow Method for Optimal k for {feature}')
    ax.axvline(x=elbow_point, c="red")
    ax.set_xticks(k_values) """

    return k, k_values, inertia

def plot_cluster(df: pd.DataFrame, feature:str, k_values: np.ndarray, inertia: np.ndarray, k: int):
    fig, axs = plt.subplots(2, figsize=(8, 6))
    axs[0].plot(k_values, inertia, marker='o')
    axs[0].set(xlabel='Number of clusters (k)', ylabel='Inertia', title=f'Elbow Method for Optimal k for {feature}')
    axs[0].axvline(x=k, c="red")
    axs[0].set_xticks(k_values)

    axs[1].scatter(df[feature], df[f'{feature}_cat'], c=df[f'{feature}_cat'], cmap='viridis')
    axs[1].set_title(f"{feature} Categories using K-Means Clustering")
    axs[1].set_xlabel(feature)
    axs[1].set_ylabel("Category")

    
CONTINUOUS_FEATURES = ["age", "bmi", "poverty"]
for feature in CONTINUOUS_FEATURES:
    k, k_values, inertia = get_optimal_cluster_number(df, feature, 10)
    kmeans = KMeans(n_clusters=k, n_init=5, random_state=42)
    df[f"{feature}_cat"] = kmeans.fit_predict(df[[feature]])

In [346]:
# regrouping

"""
EDUCATION
00 (no edu), 01 (no hs), 02 (no diploma): no_hs
03 (ged), 04 (hs), 05 (no deg): highschool
06 (occupational deg), 07 (academic deg), 08 (bsc): bachelor
09 (msc): masters
10 (phd): phd
"""

"""
MARITAL status:
03: single
01 (married) 2 (cohabiting): not
"""

"""
parent
01: parent
02 (in famility but not) 03 (not in family): not
"""


def invert_encoding_order(df: pd.DataFrame, feature: str) -> None:
    """Reorders the encoding of a categorical variable, 
    so that the lowest values are encoded 0 and higher ones ascending from here."""
    df[feature] = df[feature].max() - df[feature]

def offset_encoding_to_zero(df: pd.DataFrame, feature: str) -> None:
    """Re-encodes `1` and `2` category labels into `0` and `1`"""
    df[feature] = df[feature] - 1


offset_encoding_to_zero(df, "gender") 

class EduLevel(IntEnum):
    NO_HIGHSCHOOL = 0
    HIGHSCHOOL = 1
    BACHELORS = 2
    MASTERS = 3
    PHD = 4
    
df["education"] = df["education"].replace({
    1: EduLevel.NO_HIGHSCHOOL,
    2: EduLevel.NO_HIGHSCHOOL,
    3: EduLevel.HIGHSCHOOL,
    4: EduLevel.HIGHSCHOOL,
    5: EduLevel.HIGHSCHOOL,
    6: EduLevel.BACHELORS,
    7: EduLevel.BACHELORS,
    8: EduLevel.BACHELORS,
    9: EduLevel.MASTERS,
    10: EduLevel.PHD
})

df["single"] = df["single"].replace({
    1: 0,
    2: 0,
    3: 1
})

df["owns_home"] = df["owns_home"].replace({
    1: 1,
    2: 0,
    3: 0,
})

df["parent"] = df["parent"].replace({
    1: 1,
    2: 0,
    3: 0
})


invert_encoding_order(df, "depression")

invert_encoding_order(df, "anxiety")

invert_encoding_order(df, "health_status")

invert_encoding_order(df, "life_sat")

offset_encoding_to_zero(df, "insurance") 

invert_encoding_order(df, "living_area")

offset_encoding_to_zero(df, "region") 

invert_encoding_order(df, "cancer")

invert_encoding_order(df, "hypertension")

invert_encoding_order(df, "cholesterol")

invert_encoding_order(df, "asthma")

offset_encoding_to_zero(df, "race") 

invert_encoding_order(df, "medical_bill_worry")

df["age_cat"] = df["age_cat"].replace({
    1: 0,
    2: 1,
    0: 2,
})

df["bmi_cat"] = df["bmi_cat"].replace({
    2: 0,
    0: 1,
    1: 2,
})

### Drop continuous data

In [347]:
df.drop(CONTINUOUS_FEATURES, inplace=True, axis=1)

## Crosstables

In [348]:
""" target = "anxiety"

for f in df.columns.values:
    tab = pd.crosstab(df[f], df[target], normalize="index") * 100
    plt.figure(figsize=(6, 3))
    sns.heatmap(tab, fmt='.2f', cmap='coolwarm', annot=True)
    plt.show() """

' target = "anxiety"\n\nfor f in df.columns.values:\n    tab = pd.crosstab(df[f], df[target], normalize="index") * 100\n    plt.figure(figsize=(6, 3))\n    sns.heatmap(tab, fmt=\'.2f\', cmap=\'coolwarm\', annot=True)\n    plt.show() '

### Chi-square tests

In [349]:
X = df.drop(["anxiety"], axis=1)
y = df['anxiety']

chi2_scores, p_values = chi2(X, y)
alpha = 0.05

feature_names = X.columns
results = pd.DataFrame({
    'feature': feature_names,
    'chi': chi2_scores,
    'p': p_values
}).sort_values(by='p', ignore_index=True)

print(results)

significant_features = results[results["p"] < alpha]["feature"].values
insignificant_features =  results[results["p"] >= alpha]["feature"].values

df.drop(insignificant_features, inplace=True, axis=1)

               feature           chi              p
0           depression  15793.186491   0.000000e+00
1              age_cat   1425.727603  1.823056e-307
2   medical_bill_worry   1251.239725  1.240938e-269
3               gender   1104.225585  9.189134e-238
4        health_status    361.803253   4.957452e-77
5               asthma    360.400786   9.956869e-77
6             life_sat    345.419779   1.709514e-73
7            education    198.090764   9.667944e-42
8         hypertension    178.677826   1.433765e-37
9            owns_home    155.439801   1.389187e-32
10         poverty_cat    136.038180   1.989087e-28
11              single    119.390361   7.208863e-25
12         cholesterol    107.485402   2.501311e-22
13              parent     99.974694   9.959406e-21
14              cancer     98.369083   2.187737e-20
15         living_area     66.011423   1.575264e-13
16             bmi_cat     62.934506   7.004726e-13
17                race     39.362905   5.861853e-08
18          

In [350]:
one_hot_columns = [column for column in significant_features if len(df[column].unique()) > 2] 
df = pd.get_dummies(df, columns=one_hot_columns)

## Model training

In [352]:
X = df.drop(["anxiety"], axis=1)
y = df["anxiety"]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42, stratify=y)
smote = SMOTE(random_state=42)
X_resampled, y_resampled = smote.fit_resample(X_train, y_train)

""" model = RandomForestClassifier(
    bootstrap=False,
    random_state=42,
    criterion="gini",
    max_depth=None,
    max_features="sqrt",
    min_samples_split=2,
    n_estimators=500
) """

# model = KNeighborsClassifier(n_neighbors=11 )

""" model = MLPClassifier(
    hidden_layer_sizes=(10, 8),  # Two hidden layers with 5 and 4 neurons
    activation='relu',           # Activation function for hidden layers
    solver='adam',               # Optimizer
    max_iter=250,                # Maximum iterations
    random_state=42,              # For reproducibility
    alpha = 0.0001,
    batch_size = 64,
    learning_rate='adaptive',
    learning_rate_init=0.001  
) """

#model = SVC(kernel="rbf", C=1, gamma="scale", decision_function_shape="ovr")

""" model = XGBClassifier(
    random_state = 42,
    learning_rate=0.2
) """

model = HistGradientBoostingClassifier(
    random_state=42,
    max_depth=3, 
    max_iter=400
)

model.fit(X_resampled, y_resampled)
y_pred = model.predict(X_test)
print(classification_report(y_true=y_test, y_pred=y_pred))


              precision    recall  f1-score   support

           0       0.55      0.74      0.63      1591
           1       0.44      0.44      0.44      1650
           2       0.24      0.12      0.16       678
           3       0.37      0.34      0.35       910
           4       0.57      0.48      0.52       789

    accuracy                           0.48      5618
   macro avg       0.43      0.42      0.42      5618
weighted avg       0.45      0.48      0.46      5618

